In [5]:
import numpy as np
import pandas as pd
from collections import defaultdict
import re
from datetime import datetime

print('Hello Insight.')
print('Start processing.')

def valid_date(trans_dt): # Should not assume the year field holds any particular value. So the year value should be checked.
    
    valid = 1
    
    if not len(trans_dt)==8:
        valid = 0
    else:
        if int(trans_dt[4:8])>2018 or int(trans_dt[4:8])<2013:
            valid = 0
        if int(trans_dt[0:1])>12 or int(trans_dt[0:1])<1:
            valid = 0
        if int(trans_dt[2:3])>31 or int(trans_dt[2:3])<1:
            valid = 0
    
    return valid


def valid_zip(zipcode):
    
    valid = 1
    
    if len(zipcode)<5:
        valid = 0
    
    return valid

def valid_name(name):
    
    valid = 1
    
    if len(name)<1:
        valid = 0
    
    return valid

# CMTE_ID, AMT, OTHER_ID assumed to be easily verified.

def valid_all(donation): # Include all the invalid checks.
    
    valid = 1
    
    CMTE_ID = donation[0]
    #print(CMTE_ID)
    NAME = donation[7]
    #print(NAME)
    ZIP_CODE = donation[10]
    #print(ZIP_CODE)
    #DONAR = donation[7] + '_' + donation[10][0:5]
    TRANS_DT = donation[13]
    #print(TRANS_DT)
    #YEAR = donation[13][4:8]
    TRANS_AM = donation[14]
    #print(TRANS_AM)
    OTHER = donation[15]
    #print(OTHER)
    
    if len(CMTE_ID)==0:
        valid = 0
    if len(TRANS_AM)==0 or int(TRANS_AM)<0:
        valid = 0
    if len(OTHER)>0:
        valid = 0
    
    if not valid_date(TRANS_DT):
        valid = 0
    if not valid_zip(ZIP_CODE):
        valid = 0
    if not valid_name(NAME):
        valid = 0
    
    #print(valid_date(TRANS_DT))
    #print(valid_zip(ZIP_CODE))
    #print(valid_name(NAME))
    
    return valid

def order_time(donors_val):    # Delete the first donor and return all the repeat donor in a list.
    donors_ordered = []
    years = set()

    for donor_val in donors_val:
        years.add(int(donor_val['YEAR']))

    year_first = min(years)

    for donor_val in donors_val:
        if int(donor_val['YEAR'])>year_first:
            donors_ordered.append(donor_val)

    return donors_ordered

Hello Insight.
Start processing.


In [6]:
if __name__ == '__main__':
    #NUM = 10 # Num of lines to test on
    #i = 0

    hash_donars = defaultdict(list)
    #hash_zips = defaultdict(list)
    zips = defaultdict(set)

    with open('../input/itcont.txt') as f:

        try:

            while True:
                line = next(f)
                donation = line.split('|')

                if valid_all(donation):
                    #pass
                #else:
                    #print(i)
                    #print(line)
                    donation_dict = dict()
                    donation_dict['CMTE_ID'] = donation[0]
                    donation_dict['NAME'] = donation[7]
                    donation_dict['ZIP_CODE'] = donation[10][0:5]
                    donation_dict['DONAR'] = donation[7] + '_' + donation[10][0:5]
                    donation_dict['TRANS_DT'] = donation[13]
                    donation_dict['YEAR'] = donation[13][4:8]
                    donation_dict['TRANS_AM'] = donation[14]
                    # dona['OTHER_ID'] = donation[15]

                    #i += 1
                    #print(dona)

                    hash_donars[donation_dict['DONAR']].append(donation_dict)
                    #hash_zips[dona['ZIP_CODE']].append(dona)
                    zips[donation_dict['ZIP_CODE']].add(donation[7] + '_' + donation[10][0:5])

        except StopIteration:

            pass

    #print(hash_donars)
    #print(hash_zips)
    #print(zips)

    with open('./output.txt', 'a') as out:

        # Zip - Donars - Donors

        for zip_id in zips: # For each zip code
            i = 0
            amount = 0
            donars_zip= zips[zip_id]

            for donars in donars_zip: # For each donars inside each zip code

                if len(hash_donars[donars])>1:
                  #  pass
                #else:
                    donors_ordered = order_time(hash_donars[donars])    # Only return the donors with repeat years. NO first year.
                    #print(donors_ordered)
                    #i = 1
                    for donors in donors_ordered:  # For each repeat donor inside each donars
                        #print(donors)

                        i += 1
                        amount += int(donors['TRANS_AM'])

                        repeat_donor = '|'.join([donors['CMTE_ID'],donors['ZIP_CODE'],donors['TRANS_DT'],str(amount),str(i)])

                        print(repeat_donor)
                        out.write(repeat_donor)
                        out.write('\n')


    #print(zips)
    #print(hash_donars)


In [19]:
donation_dict = dict()
donation_dict['CMTE_ID'] = donation[0]
donation_dict['NAME'] = donation[7]
donation_dict['ZIP_CODE'] = donation[10][0:5]
donation_dict['DONAR'] = donation[7] + '_' + donation[10][0:5]
donation_dict['TRANS_DT'] = donation[13]
donation_dict['YEAR'] = donation[13][4:8]
donation_dict['TRANS_AM'] = donation[14]

In [20]:
donation_dict

{'CMTE_ID': 'C00003418',
 'DONAR': 'ELFNER, ALBERT MR. III_02108',
 'NAME': 'ELFNER, ALBERT MR. III',
 'TRANS_AM': '250',
 'TRANS_DT': '10282014',
 'YEAR': '2014',
 'ZIP_CODE': '02108'}

In [4]:
hash_donars[donars]

[{'CMTE_ID': 'C00365031',
  'DONAR': 'BUSTAMANTE, BRUCE_99501',
  'NAME': 'BUSTAMANTE, BRUCE',
  'TRANS_AM': '750',
  'TRANS_DT': '04302014',
  'YEAR': '2014',
  'ZIP_CODE': '99501'},
 {'CMTE_ID': 'C00365031',
  'DONAR': 'BUSTAMANTE, BRUCE_99501',
  'NAME': 'BUSTAMANTE, BRUCE',
  'TRANS_AM': '250',
  'TRANS_DT': '',
  'YEAR': '',
  'ZIP_CODE': '99501'}]